In [1]:
import wave
import numpy as np
import time
import matplotlib.pyplot as plt
copyright = "MrCrimson "+time.ctime()+" USTC"

fp = wave.open("test_audio.wav",'rb')

num_frames = fp.getnframes()
num_channels,sampwidth,framerate,num_frames,comptype,compname = fp.getparams()# 获得所有参数，通道数，比特宽度（每一帧的字节数），帧率，帧数，压缩格式，压缩名字


# 获得所有帧(二进制比特流)并转换为16b整型数
b_audio = fp.readframes(num_frames)
wavedata = np.fromstring(b_audio,dtype=np.uint16)
wavedata = wavedata*1.0/(max(abs(wavedata)))
wavedata = np.reshape(wavedata,[num_frames,num_channels])
fp.close()
# time = np.arange(0,num_frames)*(1.0 / framerate)
# plt.plot(time,wavedata[:,0])
# plt.xlabel("Time(s)")
# plt.ylabel("Amplitude")
# plt.title("Single channel wavedata")
# plt.grid('on')
# for i in wavedata:
#     print(i)
# print(b_audio)
def str2bin(text):
    return ''.join(format(ord(c),'b') for c in text)

# b_cpr = str2bin(copyright)

# 不知道为什么前面有很长一段都是0，可能和格式有关
# for i,b in enumerate(b_audio):
#     print(b)
#     if b != 0:
#         print(i)
#         flag = i
#         break

# # 这里读入时是每两个字节为一帧，刚好8位
# for i in range(0,len(b_cpr)):
#     tmp_high = b_audio[flag+i] 
#     print(tmp_high)



C:\Windows\Temp\ipykernel_57640\3385168763.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  wavedata = np.fromstring(b_audio,dtype=np.uint16)


In [11]:
import wave         
import struct
import sys



def lsb_watermark(cover_filepath, watermark_data, watermarked_output_path):
    
    watermark_str = str(watermark_data)
    watermark = struct.unpack("%dB" % len(watermark_str), watermark_str)
    
    watermark_size = len(watermark)
    watermark_bits = watermark_to_bits((watermark_size,), 32)
    watermark_bits.extend(watermark_to_bits(watermark))
    
    cover_audio = wave.open(cover_filepath, 'rb') 
    
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = cover_audio.getparams()
    frames = cover_audio.readframes (nframes * nchannels)
    samples = struct.unpack_from ("%dh" % nframes * nchannels, frames)

    if len(samples) < len(watermark_bits):
        raise OverflowError("The watermark data provided is too big to fit into the cover audio! Tried to fit %d bits into %d bits of space." % (len(watermark_bits), len(samples))) 
    
    print("Watermarking %s (%d samples) with %d bits of information." % (cover_filepath, len(samples), len(watermark_bits)))
    
    encoded_samples = []
    
    watermark_position = 0
    n = 0
    for sample in samples:
        encoded_sample = sample
        
        if watermark_position < len(watermark_bits):
            encode_bit = watermark_bits[watermark_position]
            if encode_bit == 1:
                encoded_sample = sample | encode_bit
            else:
                encoded_sample = sample
                if sample & 1 != 0:
                    encoded_sample = sample - 1
                    
            watermark_position = watermark_position + 1
            
        encoded_samples.append(encoded_sample)
            
    encoded_audio = wave.open(watermarked_output_path, 'wb')
    encoded_audio.setparams( (nchannels, sampwidth, framerate, nframes, comptype, compname) )

    encoded_audio.writeframes(struct.pack("%dh" % len(encoded_samples), *encoded_samples))

def watermark_to_bits(watermark, nbits=8):
    watermark_bits = []
    for byte in watermark:
        for i in range(0,nbits):
            watermark_bits.append( (byte & (2 ** i)) >> i )
    return watermark_bits
    
def recover_lsb_watermark(watermarked_filepath):
    # Simply collect the LSB from each sample
    watermarked_audio = wave.open(watermarked_filepath, 'rb') 
    
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = watermarked_audio.getparams()
    frames = watermarked_audio.readframes (nframes * nchannels)
    samples = struct.unpack_from ("%dh" % nframes * nchannels, frames)
    
  
    watermark_bytes = 0
    for (sample,i) in zip(samples[0:32], range(0,32)):
        watermark_bytes = watermark_bytes + ( (sample & 1) * (2 ** i))
    
       
    watermark_data = []
    
    for n in range(0, watermark_bytes):
        watermark_byte_samples = samples[32 + (n * 8) : 32+((n+1) * 8)]
        watermark_byte = 0
        for (sample, i) in zip(watermark_byte_samples, range(0,8)):
            watermark_byte = watermark_byte + ( (sample & 1) * (2**i) )
        watermark_data.append(watermark_byte)
            
    return watermark_data
    
def watermark_to_string(list):
    return "".join([chr(x) for x in list])

def embed_file(cover_audio, hidden_file, output):
	f = open(hidden_file)
	hidden_data = f.read()
	lsb_watermark(cover_audio, hidden_data, output)

def recover_embedded_file(encoded_signal, hidden_data_dest):
	wm = recover_lsb_watermark(encoded_signal)
	wm_str = watermark_to_string(wm)
	f = open(hidden_data_dest,"w")
	f.write(wm_str)
    
if __name__ == "__main__":
    cover_audio = "./test_audio.wav"
    output = "res.wav"
    if len(sys.argv) > 1:
        message = sys.argv[1]
        if len(sys.argv) > 2:
            cover_audio = sys.argv[2]
            if len(sys.argv) > 3:
                output = sys.argv[3]
    message = bytes(202207012344)
    print(message,type(message))
    lsb_watermark(cover_audio, message, output)
    
    recover_lsb_watermark(output)

TypeError: a bytes-like object is required, not 'str'